### importing the library

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

### Loading the dataset

In [2]:
raw_data = pd.read_csv('heart.csv')
raw_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [3]:
raw_data.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [4]:
raw_data.rename(columns ={'age':'Age','sex':'Sex','cp':'Chest_pain','trestbps':'Resting_blood_pressure','chol':'Cholesterol','fbs':'Fasting_blood_sugar',
                    'restecg':'ECG_results','thalach':'Maximum_heart_rate','exang':'Exercise_induced_angina','oldpeak':'ST_depression','ca':'Major_vessels',
                   'thal':'Thalassemia_types','target':'Heart_attack','slope':'ST_slope'}, inplace = True)

### Adding the dummy variables to data to improve the accuracy

In [5]:
dummy1 = pd.get_dummies(raw_data.Chest_pain)
dummy2 = pd.get_dummies(raw_data.Thalassemia_types)
dummy3 = pd.get_dummies(raw_data.ECG_results)
dummy4 = pd.get_dummies(raw_data.ST_slope)
dummy5 = pd.get_dummies(raw_data.Major_vessels)
# merge the dummy variables to the dataset
merge = pd.concat([raw_data,dummy1,dummy2,dummy3,dummy4,dummy5],axis = 'columns')

In [6]:
final_dataset = merge.drop(['Chest_pain','Thalassemia_types','ECG_results','ST_slope','Major_vessels'],axis = 1)
final_dataset.head()

,Age,Sex,Resting_blood_pressure,Cholesterol,Fasting_blood_sugar,Maximum_heart_rate,Exercise_induced_angina,ST_depression,Heart_attack,0,...,1,2,0,1,2,0,1,2,3,4
0,63,1,145,233,1,150,0,2.3,1,0,...,0,0,1,0,0,1,0,0,0,0
1,37,1,130,250,0,187,0,3.5,1,0,...,1,0,1,0,0,1,0,0,0,0
2,41,0,130,204,0,172,0,1.4,1,0,...,0,0,0,0,1,1,0,0,0,0
3,56,1,120,236,0,178,0,0.8,1,0,...,1,0,0,0,1,1,0,0,0,0
4,57,0,120,354,0,163,1,0.6,1,1,...,1,0,0,0,1,1,0,0,0,0


In [7]:
x = final_dataset.drop(['Heart_attack'], axis=1)
y = final_dataset['Heart_attack']

In [8]:
# splitting the dataset into training set and testing set 
x_train, x_test, y_train, y_test = train_test_split(x, y ,test_size=0.2, random_state=32)

### features scaling 
which improve the accuracy of the model using minmaxscaler to leave the dummy variables unchanged

In [9]:
scaller = MinMaxScaler()
x_train = scaller.fit_transform(x_train)
x_test = scaller.transform(x_test)

### fine tuning the by using gridsearch

In [10]:
parameters = [{'C': [0.01, 0.1,0.2, 0.5, 0.75, 1, 5, 10, 15, 20], 'kernel': ['linear']}]
classifier = SVC()
grid_search = GridSearchCV(estimator=classifier,
                          param_grid=parameters,
                          scoring='neg_mean_squared_error',
                          cv=5,
                          n_jobs=-1)
grid_search.fit(x_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Parameters:", best_parameters)

Best Parameters: {'C': 0.5, 'kernel': 'linear'}


In [11]:
parameters = [{'n_estimators': [100, 250, 500, 750, 1000],
               'criterion': ['gini','entropy'],'max_features':['auto','sqrt','log2']}]
from sklearn.ensemble import RandomForestClassifier 
classifier = RandomForestClassifier()
grid_search = GridSearchCV(estimator=classifier,
                          param_grid=parameters,
                          scoring='neg_mean_squared_error',
                          cv=5,
                          n_jobs=-1)
grid_search.fit(x_train, y_train)
best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_
print("Best Parameters:", best_parameters)

Best Parameters: {'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 100}


In [14]:
svc_classifier = SVC(C=0.5, kernel='linear', random_state=32)
svc_classifier.fit(x_train,y_train)
random_forest_classifier = RandomForestClassifier(n_estimators=100, criterion='entropy', max_features='log2', random_state=32)
random_forest_classifier.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', max_features='log2',
                       random_state=32)

In [15]:
y_svc_pred = svc_classifier.predict(x_test)
y_random_forest_pred = random_forest_classifier.predict(x_test)

print ('svc classifier accuracy: {:.2f} %'.format(accuracy_score(y_test, y_svc_pred)*100))
print ('random forest classifier accuracy: {:.2f} %'.format(accuracy_score(y_test, y_random_forest_pred)*100))

svc classifier accuracy: 83.61 %
random forest classifier accuracy: 81.97 %
